# Neural Network

Load in Data

In [1]:
import pandas as pd
%config InlineBackend.figure_formats = ['svg']

In [2]:
df_train = pd.read_pickle("training_data_final2.pkl")
df_train.head()

,game_ID,ID,blue_ping,blue_pos_x,blue_pos_y,blue_pos_z,blue_vel_x,blue_vel_y,blue_vel_z,blue_handbrake,...,blue_to_ball_vel_x,blue_to_ball_vel_y,blue_to_ball_vel_z,orange_to_ball_pos_x,orange_to_ball_pos_y,orange_to_ball_pos_z,orange_to_ball_vel_x,orange_to_ball_vel_y,orange_to_ball_vel_z,game_next_goal
0,0,479,0.004004,0.0,-0.76791,0.008318,0.0,0.018696,0.000035,False,...,0.0,0.018696,0.000035,0.0,0.76791,-0.037032,0.0,-0.018696,0.000035,0.0
1,0,481,0.004004,0.0,-0.76791,0.008318,0.0,0.018696,0.000035,False,...,0.0,0.018696,0.000035,0.0,0.76791,-0.037032,0.0,-0.018696,0.000035,0.0
2,0,482,0.004004,0.0,-0.76791,0.008318,0.0,0.018696,0.000035,False,...,0.0,0.018696,0.000035,0.0,0.76791,-0.037032,0.0,-0.018696,0.000035,0.0
3,0,483,0.004004,0.0,-0.76791,0.008318,0.0,0.018696,0.000035,False,...,0.0,0.018696,0.000035,0.0,0.76791,-0.037032,0.0,-0.018696,0.000035,0.0
4,0,484,0.004004,0.0,-0.76791,0.008318,0.0,0.018696,0.000035,False,...,0.0,0.018696,0.000035,0.0,0.76791,-0.037032,0.0,-0.018696,0.000035,0.0


In [3]:
df_train[df_train.select_dtypes("bool").columns] = df_train.select_dtypes("bool").astype(np.int32)
df_train[df_train.select_dtypes(np.float64).columns] = df_train.select_dtypes(np.float64).astype(np.float32)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450181 entries, 0 to 1450180
Data columns (total 62 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   game_ID                    1450181 non-null  int64  
 1   ID                         1450181 non-null  int64  
 2   blue_ping                  1450181 non-null  float32
 3   blue_pos_x                 1450181 non-null  float32
 4   blue_pos_y                 1450181 non-null  float32
 5   blue_pos_z                 1450181 non-null  float32
 6   blue_vel_x                 1450181 non-null  float32
 7   blue_vel_y                 1450181 non-null  float32
 8   blue_vel_z                 1450181 non-null  float32
 9   blue_handbrake             1450181 non-null  int32  
 10  blue_ball_cam              1450181 non-null  int32  
 11  blue_boost                 1450181 non-null  float32
 12  blue_boost_active          1450181 non-null  int32  
 13  blue_dodge_a

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Input, concatenate
import tensorflow as tf

In [27]:
# initial build

def custom_cv(df = df_train):
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=0)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
        
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=25, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size = 128)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [28]:
yeet = custom_cv()
yeet

Model: "functional_77"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 5, 4)         28          input_26[0][0]                   
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 16)           176         input_25[0][0]                   
______________________________________________________________________________________

(0.6745114326477051,
 array([0.66264981, 0.66538012, 0.69513232, 0.67488348]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x000002228D30D2E0>,
       dtype=object))

In [29]:
# added units to dense layer for inputB

def custom_cv2(df = df_train):
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(128, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=0)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
        
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=25, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size = 128)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [30]:
yeet_b = custom_cv2()
yeet_b

Model: "functional_101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_33 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 5, 4)         28          input_34[0][0]                   
__________________________________________________________________________________________________
dense_48 (Dense)                (None, 128)          1408        input_33[0][0]                   
_____________________________________________________________________________________

(0.6828631460666656,
 array([0.67239517, 0.67828006, 0.6919511 , 0.68882626]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x000002228F20A610>,
       dtype=object))

In [31]:
# reverted dense layer for inputB, added units to conv layer

def custom_cv3(df = df_train):
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(32, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=0)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
        
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=25, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=128)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [32]:
yeet_c = custom_cv3()
yeet_c

Model: "functional_125"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_41 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 5, 32)        224         input_42[0][0]                   
__________________________________________________________________________________________________
dense_60 (Dense)                (None, 16)           176         input_41[0][0]                   
_____________________________________________________________________________________

(0.701724037528038,
 array([0.6847958 , 0.69164717, 0.7226249 , 0.70782828]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222917E2340>,
       dtype=object))

In [33]:
# reverted conv layer, reduced lr

def custom_cv4(df = df_train):
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=0)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=25, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=128)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [34]:
yeet4 = custom_cv4()
yeet4

Model: "functional_149"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_49 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, 5, 4)         28          input_50[0][0]                   
__________________________________________________________________________________________________
dense_72 (Dense)                (None, 16)           176         input_49[0][0]                   
_____________________________________________________________________________________

(0.6602091193199158,
 array([0.65022117, 0.65597945, 0.66666883, 0.66796702]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x000002228F3EB970>,
       dtype=object))

In [35]:
# reduced lr again, increased batch size, increased epochs

def custom_cv5(df = df_train):
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=0)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00005), metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=50, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [36]:
%%time

yeet5 = custom_cv5()
yeet5

Model: "functional_173"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_57 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_28 (Conv1D)              (None, 5, 4)         28          input_58[0][0]                   
__________________________________________________________________________________________________
dense_84 (Dense)                (None, 16)           176         input_57[0][0]                   
_____________________________________________________________________________________

(0.6608124226331711,
 array([0.65234715, 0.65161753, 0.67320359, 0.66608143]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222985190A0>,
       dtype=object))

In [47]:
# increased batch size, increased epochs

def custom_cv6(df = df_train):
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.000005), metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=150, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2**15, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [48]:
%%time

yeet6 = custom_cv6()
yeet6

Model: "functional_221"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_74 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_36 (Conv1D)              (None, 5, 4)         28          input_74[0][0]                   
__________________________________________________________________________________________________
dense_108 (Dense)               (None, 16)           176         input_73[0][0]                   
_____________________________________________________________________________________

KeyboardInterrupt: 

In [56]:
# testing turned batch size up, increased lr

def custom_cv7(df = df_train):
    inputA = Input(shape=(10))
    inputB = Input(shape=(5, 6))

    a = Dense(16, activation='relu')(inputA)
    a = Model(inputs=inputA, outputs=a)

    b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
    b = Flatten()(b)
    b = Model(inputs=inputB, outputs=b)

    combined = concatenate([a.output, b.output])

    c = Dense(units=2**8, activation='relu')(combined)
    c = Dense(units=1, activation='sigmoid')(c)

    model = Model(inputs=[a.input, b.input], outputs=c)

    print(model.summary())

    checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
    
    
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        model.fit([X1, X2], y, epochs=150, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2**15, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [57]:
%%time

yeet7 = custom_cv7()
yeet7

Model: "functional_247"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_86 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_85 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_42 (Conv1D)              (None, 5, 4)         28          input_86[0][0]                   
__________________________________________________________________________________________________
dense_120 (Dense)               (None, 16)           176         input_85[0][0]                   
_____________________________________________________________________________________

(0.6613594442605972,
 array([0.6634717 , 0.66012758, 0.66453475, 0.65730375]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222A0561E80>,
       dtype=object))

In [58]:
# reverted batch size, # lowered dense units after concat

def custom_cv8(df = df_train):
    inputA = Input(shape=(10))
    inputB = Input(shape=(5, 6))

    a = Dense(16, activation='relu')(inputA)
    a = Model(inputs=inputA, outputs=a)

    b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
    b = Flatten()(b)
    b = Model(inputs=inputB, outputs=b)

    combined = concatenate([a.output, b.output])

    c = Dense(units=2**6, activation='relu')(combined)
    c = Dense(units=1, activation='sigmoid')(c)

    model = Model(inputs=[a.input, b.input], outputs=c)

    print(model.summary())

    checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
    
    
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        model.fit([X1, X2], y, epochs=150, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [59]:
%%time

yeet8 = custom_cv8()
yeet8

Model: "functional_253"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_88 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_87 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_43 (Conv1D)              (None, 5, 4)         28          input_88[0][0]                   
__________________________________________________________________________________________________
dense_123 (Dense)               (None, 16)           176         input_87[0][0]                   
_____________________________________________________________________________________

(0.6514025777578354,
 array([0.65198964, 0.64910543, 0.65778452, 0.64673072]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222A28CE520>,
       dtype=object))

In [65]:
# fixed bug with iterations using previous best log loss

def custom_cv9(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=(10))
        inputB = Input(shape=(5, 6))

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**6, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)

        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
        
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [66]:
%%time

yeet9 = custom_cv9()
yeet9

Model: "functional_277"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_96 (InputLayer)           [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_95 (InputLayer)           [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_47 (Conv1D)              (None, 5, 4)         28          input_96[0][0]                   
__________________________________________________________________________________________________
dense_135 (Dense)               (None, 16)           176         input_95[0][0]                   
_____________________________________________________________________________________

(0.6636629998683929,
 array([0.65006298, 0.65886873, 0.67688006, 0.66884023]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x000002229A207C70>,
       dtype=object))

In [67]:
# reduced units in dense layer after concat

def custom_cv10(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=(10))
        inputB = Input(shape=(5, 6))

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**5, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)

        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
        
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [70]:
yeet10 = custom_cv10()
yeet10

Model: "functional_313"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_108 (InputLayer)          [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_107 (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_53 (Conv1D)              (None, 5, 4)         28          input_108[0][0]                  
__________________________________________________________________________________________________
dense_153 (Dense)               (None, 16)           176         input_107[0][0]                  
_____________________________________________________________________________________

(0.6619095057249069,
 array([0.65365177, 0.65697801, 0.66860896, 0.66839927]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222A7A4CE20>,
       dtype=object))

In [73]:
# changed optimizer to sgd

def custom_cv11(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=(10))
        inputB = Input(shape=(5, 6))

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**5, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)

        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.01), metrics=['accuracy'])
        
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [74]:
yeet11 = custom_cv11()
yeet11

Model: "functional_343"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_118 (InputLayer)          [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_117 (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_58 (Conv1D)              (None, 5, 4)         28          input_118[0][0]                  
__________________________________________________________________________________________________
dense_168 (Dense)               (None, 16)           176         input_117[0][0]                  
_____________________________________________________________________________________

(0.6633226275444031,
 array([0.64742082, 0.65621936, 0.67596108, 0.67368925]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222A769D7C0>,
       dtype=object))

In [76]:
# increased units in dense layer after concat, increased patience

def custom_cv12(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=(10))
        inputB = Input(shape=(5, 6))

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**7, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)

        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)  
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.01), metrics=['accuracy'])
        
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [77]:
yeet12 = custom_cv12()
yeet12

Model: "functional_367"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_126 (InputLayer)          [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_125 (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_62 (Conv1D)              (None, 5, 4)         28          input_126[0][0]                  
__________________________________________________________________________________________________
dense_180 (Dense)               (None, 16)           176         input_125[0][0]                  
_____________________________________________________________________________________

(0.6614851355552673,
 array([0.65543491, 0.656124  , 0.67143393, 0.66294771]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222AADBCF70>,
       dtype=object))

In [78]:
# slightly decreased units in dense layer after concat, increased lr

def custom_cv13(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=(10))
        inputB = Input(shape=(5, 6))

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**6, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)

        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)  
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.025), metrics=['accuracy'])
        
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [79]:
%%time

yeet13 = custom_cv13()
yeet13

Model: "functional_391"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_134 (InputLayer)          [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_133 (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_66 (Conv1D)              (None, 5, 4)         28          input_134[0][0]                  
__________________________________________________________________________________________________
dense_192 (Dense)               (None, 16)           176         input_133[0][0]                  
_____________________________________________________________________________________

(0.6638134717941284,
 array([0.65134126, 0.66073602, 0.67625284, 0.66692376]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222DC966130>,
       dtype=object))

In [80]:
# slightly decreased units in dense layer after concat, slightly decreased lr

def custom_cv14(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=(10))
        inputB = Input(shape=(5, 6))

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**5, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)

        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)  
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.02), metrics=['accuracy'])
        
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=2048, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [81]:
%%time

yeet14 = custom_cv14()
yeet14

Model: "functional_415"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_142 (InputLayer)          [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_141 (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_70 (Conv1D)              (None, 5, 4)         28          input_142[0][0]                  
__________________________________________________________________________________________________
dense_204 (Dense)               (None, 16)           176         input_141[0][0]                  
_____________________________________________________________________________________

(0.6625548601150513,
 array([0.65340102, 0.65802383, 0.67695707, 0.66183752]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222B1E4AC40>,
       dtype=object))

In [86]:
# reverted to cv4, increased lr and patience

def custom_cv15(df = df_train): 
    loglosses = np.empty([0, 0])
    mods = np.empty([0, 0])
    for i in range(4):
        df_val = df.loc[(df['game_ID'] >= i*20) & (df['game_ID'] < (i+1)*20), ].copy()
        
        df_train = df.loc[(df['game_ID'] < i*20) | (df['game_ID'] >= (i+1)*20), ].copy()
        
        X1 = df_train[["blue_ping", 
              "blue_boost", 
              "blue_possession", 
              "blue_dead", 
              "orange_ping", 
              "orange_boost", 
              "orange_possession", 
              "orange_dead", 
              "into_blue_net", 
              "into_orange_net"]].copy()

        XV1 = df_val[["blue_ping", 
                      "blue_boost", 
                      "blue_possession", 
                      "blue_dead", 
                      "orange_ping", 
                      "orange_boost", 
                      "orange_possession", 
                      "orange_dead", 
                      "into_blue_net", 
                      "into_orange_net"]].copy()

        X2 = np.array([ df_train[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_train[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_train[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_train[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_train[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        X2 = np.swapaxes(X2, 0, 1)

        XV2 = np.array([df_val[["blue_pos_x", "blue_pos_y", "blue_pos_z", 
                           "blue_vel_x", "blue_vel_y", "blue_vel_z"]], 
                       df_val[["blue_to_ball_pos_x", "blue_to_ball_pos_y", "blue_to_ball_pos_z", 
                           "blue_to_ball_vel_x", "blue_to_ball_vel_y", "blue_to_ball_vel_z"]],  
                       df_val[["orange_pos_x", "orange_pos_y", "orange_pos_z", 
                           "orange_vel_x", "orange_vel_y", "orange_vel_z"]], 
                       df_val[["orange_to_ball_pos_x", "orange_to_ball_pos_y", "orange_to_ball_pos_z", 
                           "orange_to_ball_vel_x", "orange_to_ball_vel_y", "orange_to_ball_vel_z"]],  
                       df_val[["ball_pos_x", "ball_pos_y", "ball_pos_z", 
                           "ball_vel_x", "ball_vel_y", "ball_vel_z"]] ])

        XV2 = np.swapaxes(XV2, 0, 1)

        y = df_train["game_next_goal"].copy()

        yv = df_val["game_next_goal"].copy()
        
        inputA = Input(shape=X1.shape[1:])
        inputB = Input(shape=X2.shape[1:])

        a = Dense(16, activation='relu')(inputA)
        a = Model(inputs=inputA, outputs=a)

        b = Conv1D(4, kernel_size=1, activation='relu')(inputB)
        b = Flatten()(b)
        b = Model(inputs=inputB, outputs=b)

        combined = concatenate([a.output, b.output])

        c = Dense(units=2**8, activation='relu')(combined)
        c = Dense(units=1, activation='sigmoid')(c)

        model = Model(inputs=[a.input, b.input], outputs=c)
        
        print(model.summary())
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model6', save_best_only=True, verbose=1)
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
        
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
        model.fit([X1, X2], y, epochs=200, validation_data=([XV1, XV2], yv), 
                  callbacks=[stop_early, checkpoint], batch_size=128, verbose=0)
        
        mod = tf.keras.models.load_model('best_model6')
        ll = mod.evaluate([XV1, XV2], yv)[0]
        
        loglosses = np.append(loglosses, ll)
        mods = np.append(mods, mod)
        
        print(f"cv: {i}, logloss: {ll}")
    
    return (loglosses.mean(), loglosses, mods)

In [87]:
%%time

yeet15 = custom_cv15()
yeet15

Model: "functional_451"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_154 (InputLayer)          [(None, 5, 6)]       0                                            
__________________________________________________________________________________________________
input_153 (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv1d_76 (Conv1D)              (None, 5, 4)         28          input_154[0][0]                  
__________________________________________________________________________________________________
dense_222 (Dense)               (None, 16)           176         input_153[0][0]                  
_____________________________________________________________________________________

(0.660720482468605,
 array([0.65189457, 0.6557585 , 0.67483801, 0.66039085]),
 array([<tensorflow.python.keras.engine.functional.Functional object at 0x00000222E3F83AF0>,
       dtype=object))

In [88]:
mod = yeet15[2][0]
mod

In [89]:
mod.save('neural_network')

INFO:tensorflow:Assets written to: neural_network\assets
